In [195]:
# importing some useful libraries for the first part of project
import numpy as np
import scipy
import pandas as pd
import math
import random
import pickle
import seaborn as sns

from sklearn.multioutput import RegressorChain
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt

In [198]:
df = pd.read_csv('/content/Paper3_WebData_Final.csv')
print(df.shape)
df = pd.read_csv('Paper3_WebData_Final.csv')
df = df.drop(['test_date','nid','gender','naics', 'age_group','region','NAICS_descr'], axis=1)
df_L = df.drop(['R500k','R1k','R2k','R3k', 'R4k', 'R6k', 'R8k'], axis=1)
df_R = df.drop(['L500k','L1k','L2k','L3k', 'L4k', 'L6k', 'L8k'], axis=1)

# Seperating left and right ears data so that we could use them for creating model.
df_L.columns = ['500k','1k','2k','3k', '4k', '6k', '8k']
df_R.columns = ['500k','1k','2k','3k', '4k', '6k', '8k']
data = pd.concat([df_L, df_R], axis=0)
data =data.reset_index(drop=True)
# showing data
print(data.shape)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1831084, 21)
(3776780, 7)


In [199]:
# cleaning data, removing nan values, and '**', and converting object type to int.
data.isnull().sum()/data.shape[0]
data = data.replace('**',np.nan)
data = data.dropna()
data.isnull().sum()
data=data.astype(float)

# showing data
print(data.shape)

(2567392, 7)


In [200]:
df = data.copy()
df =df.drop_duplicates()
print(df.shape)

(661787, 7)


In [202]:
# Based on the previous notebook, I will predict '3k', and so on.( based on col_list), I chose this way based on heatmap correlation.
print(df.columns)
col_list = ['3k', '8k', '1k', '500k']
print(col_list)

Index(['500k', '1k', '2k', '3k', '4k', '6k', '8k'], dtype='object')
['3k', '8k', '1k', '500k']


In [206]:
# defining a class in order to take care of data in each step, there are 4 steps.
# Here with this class you could change the order of prediction, for instance you could have col_list= ['8k', '500k', '1k', '3k'], and
# then predict '8k', and'500k', and so on
class RemoveDuplicate():
    d_list =[]
    def __init__(self, data, col_list, column):
        self.column = column
        self.col_list = col_list
        self.data = data
    

    def create_data(self):
    
        data1 = self.data.copy()
        for d in self.col_list:
            
            #print(data1.shape)
            df = data1.drop(d,axis=1)
            df =df.drop_duplicates()
            val_index = list(df.index.values.tolist())
            self.d_list.append(val_index)
            data1 =df
        
        if self.column == self.col_list[3]:
          #print(len(self.d_list[0]))
          dX = self.data.loc[self.d_list[0]]
          X = dX.drop(columns=[self.col_list[3]], axis=1)
          y = dX[self.col_list[3]]
          X = X.to_numpy()
          y = y.to_numpy()
          y = y.flatten()
          #print(X.shape,y.shape)
          return X,y


        if self.column == self.col_list[2]:
          #print(len(self.d_list[1]))
          dX = self.data.loc[self.d_list[1]]
          X = dX.drop(columns=[self.col_list[3], self.col_list[2] ], axis=1)
          y = dX[self.col_list[2]]
          X = X.to_numpy()
          y = y.to_numpy()
          y = y.flatten()
          #print(X.shape,y.shape)
          
          return X,y
        
        if self.column == self.col_list[1]:
          #print(len(self.d_list[2]))
          dX = self.data.loc[self.d_list[2]]
          X = dX.drop(columns=[self.col_list[3], self.col_list[2], self.col_list[1] ], axis=1)
          y = dX[self.col_list[1]]
          X = X.to_numpy()
          y = y.to_numpy()
          y = y.flatten()
          #print(X.shape,y.shape)
          return X,y
        
        if self.column == self.col_list[0]:
          #print(len(self.d_list[3]))
          dX = self.data.loc[self.d_list[3]]
          X = dX.drop(columns=[self.col_list[3], self.col_list[2], self.col_list[1], self.col_list[0] ], axis=1)
          y = dX['3k']
          X = X.to_numpy()
          y = y.to_numpy()
          y = y.flatten()
          #print(X.shape,y.shape)
          return X,y

In [208]:
# For instance , if somebody wants to predict '3k', one should remove all three columns.
# This class gives X, y datset for each step.
RemoveDuplicate(df, col_list, '3k').create_data()


(array([[ 5., 10.,  5.],
        [ 5.,  5., 10.],
        [20., 15., 20.],
        ...,
        [25., 90., 99.],
        [45., 99., 99.],
        [25., 65., 99.]]), array([15., 15., 20., ..., 85., 70., 50.]))

In [215]:
# Defining a function to take care of model parameters, here I used Random Forest parameters

def grid_Serch_pipeline(df, col_list, column, model, parameters):

    X, y = RemoveDuplicate(df, col_list, column).create_data()
   
    grid_search = GridSearchCV(model, param_grid=parameters, n_jobs=1)
    grid_search.fit(X, y)


    grid_search.fit(X, y)
    return grid_search.best_score_, grid_search.best_params__

In [216]:
parameters= {
    "n_estimators": [5, 10,50,100, 300, 500],
    "max_features": ["auto", "sqrt", "log2"],
    "max_depth": [1,5,10, 20, 50, 70, 80, 100, None],
    "bootstrap": [True, False],
    }

model = RandomForestRegressor()

In [219]:
#accuracy, parameters =grid_Serch_pipeline(df, col_list, '3k', model, parameters)


In [192]:
# Here someone could create the models, and save it for usage
# Here this could take a long time to finish, becasue I put the grid search for each step in this part.
# with this model you could ask more input values from a person, and predict the rest of gain frequnceis
from joblib import Parallel, delayed
import joblib

col_list = ['3k', '8k', '1k', '500k']

def ModelRF(col_list):
    
    models = []
    # i = 1
    for c, column in enumerate(col_list):
    
        accuracy, parameters =grid_Serch_pipeline(df, col_list, column)
        model, X, y = build_pipeline(df, col_list, column, parameters)
        model.fit(X,y)
        joblib.dump(model, 'RandomForest+"c".pkl')
        models.append(model)

        # pred1 = model.predict(inputs)

        # inputs=np.append(inputs,int(pred1))
        # inputs = inputs.reshape(1, 3 + i)
        # i +=1

    # inputs = inputs.reshape(7,)
    # outputs = np.array([inputs[6],  inputs[5], inputs[0], inputs[3], inputs[1], inputs[2], inputs[4]], dtype='int8')
    
    return models

In [ ]:
# Here I tested the model in colab, this is the same previous model I wrote in jupyter, but with less code (creating some pipeline)
import sys, os
import time


def resource_path(relative_path):
    """ Get the absolute path to the resource, works for dev and for PyInstaller """
    try:
        # PyInstaller creates a temp folder and stores path in _MEIPASS
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

In [ ]:
with open(resource_path("C:/Users/gjaha/pyproj/Take_Home_Project_ Machine_Learning/RandomForest0.pkl"), "rb") as f:
    forest1 = joblib.load(f)
with open(resource_path("C:/Users/gjaha/pyproj/Take_Home_Project_ Machine_Learning/RandomForest1.pkl"), "rb") as f1:
    forest2 = joblib.load(f1)
with open(resource_path("C:/Users/gjaha/pyproj/Take_Home_Project_ Machine_Learning/RandomForest2.pkl"), "rb") as f2:
    forest3 = joblib.load(f2)
with open(resource_path("C:/Users/gjaha/pyproj/Take_Home_Project_ Machine_Learning/RandomForest3.pkl"), "rb") as f3:
    forest4 = joblib.load(f3)

In [ ]:
# input three data
val_2k = int(input("Enter individuals hearing threshold gain value (in dB) at 2kHz: "))
val_4k = int(input("Enter individuals hearing threshold gain value (in dB) at 4kHz: "))
val_6k = int(input("Enter individuals hearing threshold gain value (in dB) at 6kHz: "))

In [ ]:
inputs1 = np.array([val_2k, val_4k, val_6k], dtype='int8')
out1 = forest1.predict([inputs1]).astype(int)
inputs2 = np.array([val_2k, val_4k, val_6k, out1[0]], dtype='int8')
out2 = forest2.predict([inputs2]).astype(int)
inputs3 = np.array([val_2k, val_4k, val_6k, out1[0], out2[0]], dtype='int8')
out3 = forest3.predict([inputs3]).astype(int)
inputs4 = np.array([val_2k, val_4k, val_6k, out1[0], out2[0], out3[0]], dtype='int8')
out4 = forest4.predict([inputs4]).astype(int)
output = np.array([out4[0],  out3[0], val_2k, out1[0], val_4k, val_6k, out2[0] ], dtype='int8')
output